In [3]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

def get_sp500_symbols():
    sp_500_url = 'http://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

    tables = pd.read_html(sp_500_url)
    sp_500 = tables[0]
    return sp_500['Symbol'].tolist()

get_sp500_symbols()


['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADBE',
 'AMD',
 'AES',
 'AFL',
 'A',
 'APD',
 'ABNB',
 'AKAM',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'APO',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BAX',
 'BDX',
 'BRK.B',
 'BBY',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'BK',
 'BA',
 'BKNG',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'BLDR',
 'BG',
 'BXP',
 'CHRW',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'COR',
 'CNC',
 'CNP',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CAG',
 'COP',
 'ED',
 'STZ',
 'CEG',
 'COO',


In [4]:
import yfinance as yf
from datetime import datetime , timedelta

def get_finance_data(symbols, start_date, end_date, interval):
    result = {}
    for symbol in symbols:
        data = yf.download(tickers=symbol, start=start_date, end=end_date, interval=interval)
        if not data.empty:
            result[symbol] = data
    return result

fetched_data = get_finance_data(['AAPL', 'MSFT'], '2025-04-14', '2025-04-17', '1d')
print(fetched_data)


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

{'AAPL': Price            Close        High         Low        Open     Volume
Ticker            AAPL        AAPL        AAPL        AAPL       AAPL
Date                                                                 
2025-04-14  202.520004  212.940002  201.160004  211.440002  101352900
2025-04-15  202.139999  203.509995  199.800003  201.860001   51343900
2025-04-16  194.270004  200.699997  192.369995  198.360001   59732400, 'MSFT': Price            Close        High         Low        Open    Volume
Ticker            MSFT        MSFT        MSFT        MSFT      MSFT
Date                                                                
2025-04-14  387.809998  394.649994  384.209991  393.220001  19251200
2025-04-15  385.730011  391.890015  384.160004  388.510010  17199900
2025-04-16  371.609985  381.609985  368.000000  380.670013  21967800}


In [5]:
def transform_data(data, symbol):
    data.columns = [col[0] for col in data.columns]
    print(data)
    data = data.reset_index()
    print(data)

    # data = data[data["Close"] != 0]
    data["symbol"] = symbol
    data['close_change'] = data['Close'].diff().fillna(0)
    data['close_pct_change'] = data['Close'].pct_change().fillna(0) * 100
    print(data)
    return data[['Date', 'Close', 'High', 'Low', 'Open', 'Volume', 'symbol', 'close_change', 'close_pct_change']]

def ingest_yfinance_data(symbol_data, final_table, interval):
    values = []
    for symbol in symbol_data:
        try:
            start_date = (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d')
            end_date = datetime.now().strftime('%Y-%m-%d')
            data_dict = get_finance_data([symbol], start_date, end_date, interval)
            if symbol in data_dict:
                data = transform_data(data_dict[symbol], symbol)
   
                if not data.empty:
                    values.extend([tuple(x) for x in data.to_numpy()])
        except Exception as e:
            print(e)
            # error_logger.error(f"{symbol} transformation error: {str(e)}")
    # if values:
    #     insert_data_to_db(final_table, values)
ingest_yfinance_data(['AAPL','MSFT'],"",'1d')

[*********************100%***********************]  1 of 1 completed


                 Close        High         Low        Open    Volume
Date                                                                
2025-04-21  193.160004  193.800003  189.809998  193.270004  46742500
2025-04-22  199.740005  201.589996  195.970001  196.119995  52976400
2025-04-23  204.600006  208.000000  202.800003  206.000000  52929200
2025-04-24  208.369995  208.830002  202.940002  204.889999  47311000
2025-04-25  209.279999  209.750000  206.199997  206.369995  38198600
        Date       Close        High         Low        Open    Volume
0 2025-04-21  193.160004  193.800003  189.809998  193.270004  46742500
1 2025-04-22  199.740005  201.589996  195.970001  196.119995  52976400
2 2025-04-23  204.600006  208.000000  202.800003  206.000000  52929200
3 2025-04-24  208.369995  208.830002  202.940002  204.889999  47311000
4 2025-04-25  209.279999  209.750000  206.199997  206.369995  38198600
        Date       Close        High         Low        Open    Volume symbol  \
0 2025-04-

[*********************100%***********************]  1 of 1 completed

                 Close        High         Low        Open    Volume
Date                                                                
2025-04-21  359.119995  364.480011  355.670013  362.820007  20807300
2025-04-22  366.820007  367.769989  359.859985  363.380005  19485000
2025-04-23  374.390015  380.390015  373.019989  376.059998  20545500
2025-04-24  387.299988  388.450012  375.190002  375.700012  22232300
2025-04-25  391.850006  392.160004  384.600006  387.000000  18963600
        Date       Close        High         Low        Open    Volume
0 2025-04-21  359.119995  364.480011  355.670013  362.820007  20807300
1 2025-04-22  366.820007  367.769989  359.859985  363.380005  19485000
2 2025-04-23  374.390015  380.390015  373.019989  376.059998  20545500
3 2025-04-24  387.299988  388.450012  375.190002  375.700012  22232300
4 2025-04-25  391.850006  392.160004  384.600006  387.000000  18963600
        Date       Close        High         Low        Open    Volume symbol  \
0 2025-04-

In [6]:
import pyodbc

def create_connection():
    conn_str = (
        r"DRIVER={SQL Server};"
        r"SERVER=DESKTOP-QMACISC;"
        r"DATABASE=BikeStores;"
        r"UID=sa;"
        r"PWD=123456789;"
    )

    try:
        conn = pyodbc.connect(conn_str)
        print("✅ Connected to SQL Server successfully!")
        return conn
    except Exception as e:
        print(f"❌ Error connecting to SQL Server: {e}")
        return None

def create_table_if_not_exists(table_name):
    conn = create_connection()
    if not conn:
        return

    cursor = conn.cursor()
    create_table_query = f"""
    IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='{table_name}' AND xtype='U')
    CREATE TABLE {table_name} (
        trade_date DATETIME,
        close_price FLOAT,
        high_price FLOAT,
        low_price FLOAT,
        open_price FLOAT,
        volume BIGINT,
        symbol VARCHAR(10),
        close_change FLOAT,
        close_pct_change FLOAT
    )
    """
    try:
        cursor.execute(create_table_query)
        conn.commit()
        print(f"✅ Table ensured: {table_name}")
    except Exception as e:
        print(f"❌ Error creating table: {e}")
    finally:
        cursor.close()
        conn.close()

def insert_data_to_db(table_name, values):
    conn = create_connection()
    if not conn:
        return

    cursor = conn.cursor()
    insert_query = f"""
    INSERT INTO {table_name} (
        trade_date, close_price, high_price, low_price, open_price,
        volume, symbol, close_change, close_pct_change
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    """

    try:
        cursor.executemany(insert_query, values)
        conn.commit()
        print(f"✅ Inserted {len(values)} rows into {table_name}")
    except Exception as e:
        print(f"❌ Error inserting data: {e}")
    finally:
        cursor.close()
        conn.close()

In [7]:

def get_sp500_symbols():
    sp_500_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    
    tables = pd.read_html(sp_500_url)
    sp_500 = tables[0]
    return sp_500['Symbol'].tolist()

def get_finance_data (symbols,start_date,end_date, interval):
    result ={}
    for symbol in symbols:
        data = yf.download(tickers = symbol,start=start_date,end = end_date, interval=interval )
        if not data.empty:
            result[symbol] = data
    return result 
    
def transform_data(data, symbol):
    data.columns = [col[0] for col in data.columns]
    data = data.reset_index()

    # data = data[data["Close"] != 0]
    data["symbol"] = symbol
    data['close_change'] = data['Close'].diff().fillna(0)
    data['close_pct_change'] = data['Close'].pct_change().fillna(0) * 100
    return data[['Date', 'Close', 'High', 'Low', 'Open', 'Volume', 'symbol', 'close_change', 'close_pct_change']]

def ingest_yfinance_data(symbol_data, table_name, interval):
    values = []
    for symbol in symbol_data:
        try:
            start_date = (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d')
            end_date = datetime.now().strftime('%Y-%m-%d')
            data_dict = get_finance_data([symbol], start_date, end_date, interval)
            if symbol in data_dict:
                data = transform_data(data_dict[symbol], symbol)
                if not data.empty:
                    values.extend([tuple(x) for x in data.to_numpy()])
        except Exception as e:
            print(e)
            # error_logger.error(f"{symbol} transformation error: {str(e)}")
    if values:
        insert_data_to_db(table_name, values)


def main(interval):
    table_name = "stock_prices"
    symbols = get_sp500_symbols()
    print(len(symbols))
    create_table_if_not_exists(table_name)
    ingest_yfinance_data(symbols ,table_name, interval)

In [8]:
main('1d')

503
✅ Connected to SQL Server successfully!
✅ Table ensured: stock_prices


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

✅ Connected to SQL Server successfully!
✅ Inserted 2505 rows into stock_prices
